In [3]:
import pandas as pd 
import json
import requests
from bs4 import BeautifulSoup

# Part1: Create the Dataframe

In [12]:
#import table from html as a dataframe
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
raw = pd.read_html(str(table))
df_toronto = raw[0]
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [13]:
# Get indexes for which the column Borough contains Not assigned
indexNames = df_toronto[df_toronto['Borough'] == 'Not assigned'].index
 
# Delete these row indexes from dataFrame
df_toronto.drop(indexNames, inplace = True)
df_toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [14]:
#Combine the Neighbourhoods for which in the same Borough 
df_toronto_cleaned = df_toronto.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))
df_toronto_cleaned2 = df_toronto_cleaned.reset_index()
df_toronto_cleaned2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
#Make the 'Not assigned' Neighbourhood the same as Borough
index_change = df_toronto_cleaned2.loc[df_toronto_cleaned2['Neighbourhood'] == 'Not assigned'].index
for index in index_change:
    df_toronto_cleaned2.iloc[index,2] = df_toronto_cleaned2.iloc[index,1]
df_toronto_cleaned2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
#get the dimension of the final dataframe
df_toronto_cleaned2.shape

(103, 3)

# Part 2: Add in Geographical Coordinates

In [11]:
#read the CSV file to dataframe
coor = pd.read_csv('https://cocl.us/Geospatial_data')
#rename the column name for postcode
coor.rename(columns = {'Postal Code':'Postcode'}, inplace = True)
coor.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
# Add the geographical coordinates by merging the two dataframe
df_coor = pd.merge(df_toronto_cleaned2, coor, on = 'Postcode')
df_coor.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
